# 👋 Welcome to the **Attribution Metrics** tutorial


In this tutorial, we will take a model, several images and run several **Attribution methods** to compare them **quantitatively** using Metrics.

We will download images from tiny-imagenet-200 dataset and compute explanations using a pre-trained model. Once the explanations are computed, we can test their **Faithfullness** using one of the available metrics: `Deletion`,`Insertion` or `MuFidelity`. Another property of a good explanation is **Stability**: for this we can use the `AverageStability` metric.

💡 In order to better understand these metrics, I put here some of the references:
[Deletion & Insertion](https://arxiv.org/pdf/1806.07421.pdf), [MuFidelity, AverageStability](https://arxiv.org/pdf/2005.00631.pdf).

_Author: [Thomas FEL](https://thomasfel.fr/)_

<p align="center">
  <img src="https://github.com/deel-ai/xplique/blob/master/docs/assets/banner.png?raw=true"/>
</p>




- 📘 <a href="https://deel-ai.github.io/xplique"> Documentation </a>
- <img src="https://github.githubassets.com/images/icons/emoji/octocat.png" width=20> <a href="https://github.com/deel-ai/xplique"> Github </a>
- 🦊 <a href="https://github.com/deel-ai/xplique/blob/master/TUTORIALS.md"> More Xplique tutorials </a>



# Installing **Xplique** and utils

In [2]:
!pip install -q xplique

import os
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

import xplique
from xplique.plots import plot_attributions

# 🐧 Download and load some Penguins images

In [5]:
!curl -O 'http://cs231n.stanford.edu/tiny-imagenet-200'
!unzip -qq ./tiny-imagenet-200
!rm tiny-imagenet-200
# !cd sample_data
# !pwd

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   253  100   253    0     0    282      0 --:--:-- --:--:-- --:--:--   282
[./tiny-imagenet-200]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./tiny-imagenet-200 or
        ./tiny-imagenet-200.zip, and cannot find ./tiny-imagenet-200.ZIP, period.


In [3]:
nb_samples = 50
input_size = 224

images_dir = 'tiny-imagenet-200/train/n02056570/images' # king penguin class :)

images_path = [f"{images_dir}/{p}" for p in os.listdir(images_dir)][:nb_samples]
images = [cv2.imread(p)[...,::-1] for p in images_path]
images = tf.image.resize(images, (input_size, input_size))
images.shape

plt.rcParams["figure.figsize"] = [15, 5]
for i in range(10):
  plt.subplot(1, 10, i+1)
  plt.imshow(images[i]/255.0)
  plt.axis('off')

FileNotFoundError: [Errno 2] No such file or directory: 'tiny-imagenet-200/train/n02056570/images'

In [ ]:
import tensorflow.keras.applications as app

model, preprocessing = app.MobileNetV2(), app.mobilenet_v2.preprocess_input

X = preprocessing(np.array(images, copy=True))
Y = tf.one_hot([145] * len(X), 1000) # class 145 is king penguin

# 🔍 Generate explanations

💡 To explain the logits is to explain the class, to explain the softmax is to explain why this class rather than another. It is thus recommended to remove the softmax activation to explain the logit


In [ ]:
from xplique.attributions import (Saliency, GradientInput, IntegratedGradients, SmoothGrad, VarGrad,
                                  SquareGrad, GradCAM, Occlusion, Rise, GuidedBackprop,
                                  GradCAMPP, Lime, KernelShap, SobolAttributionMethod)

# to explain the logits is to explain the class,
# to explain the softmax is to explain why this class rather than another
# it is therefore recommended to explain the logit
model.layers[-1].activation = tf.keras.activations.linear
batch_size = 64

explainers = [
             Saliency(model),
             GradientInput(model),
             GuidedBackprop(model),
             IntegratedGradients(model, steps=50, batch_size=batch_size),
             SmoothGrad(model, nb_samples=50, batch_size=batch_size),
             SquareGrad(model, nb_samples=50, batch_size=batch_size),
             VarGrad(model, nb_samples=50, batch_size=batch_size),
             GradCAM(model),
             GradCAMPP(model),
             Occlusion(model, patch_size=10, patch_stride=10, batch_size=batch_size),
             SobolAttributionMethod(model, batch_size=batch_size),
             # Rise(model, nb_samples=4000, batch_size=batch_size),
             # Lime(model, nb_samples = 1000),
             # KernelShap(model, nb_samples = 1000)
]

explanations_to_test = {}

for explainer in explainers:

  explainer_name = explainer.__class__.__name__
  explanations = explainer(X, Y)

  if len(explanations.shape) > 3:
    explanations = np.mean(explanations, -1)

  # store the explanations to use the metrics
  explanations_to_test[explainer_name] = explanations

  print(f"Method: {explainer_name}")
  plot_attributions(explanations[:5], X[:5], cmap='jet', alpha=0.4,
                    cols=5, clip_percentile=0.5, absolute_value=True)
  plt.show()
  print("\n")

# 🏅 Metrics

In [ ]:
%load_ext google.colab.data_table
import pandas as pd

# softmax or sigmoid are needed to correctly compute metrics this time
model = app.MobileNetV2()

## **Deletion** Metric

The deletion metric measures the drop in the probability of a class as important pixels (given by the saliency map) are gradually removed from the image. A sharp drop, and thus a small area under the probability curve, are indicative of a good explanation.

**Lower is better**

---

Petsiuk & al., RISE: Randomized Input Sampling for Explanation of Black-box Models (2018). https://arxiv.org/pdf/1806.07421.pdf

In [ ]:
from xplique.metrics import Deletion

metric = Deletion(model, X, Y, batch_size)
deletion_scores = []

i = 0
for explanation_name, explanations in explanations_to_test.items():
  deletion_score = metric(explanations)
  deletion_scores.append((explanation_name, deletion_score))

pd.DataFrame(deletion_scores, columns=["Method name", "Deletion score (lower is better)"])

## **Insertion** Metric

The insertion metric, on the other hand, captures the importance of the pixels in terms of their ability to synthesize an image and is measured by the rise in the probability of the class of interest as pixels are added according to the generated importance map.

**Higher is better**

---

Petsiuk & al., RISE: Randomized Input Sampling for Explanation of Black-box Models (2018). https://arxiv.org/pdf/1806.07421.pdf

In [ ]:
from xplique.metrics import Insertion

metric = Insertion(model, X, Y, batch_size)
insertion_scores = []

i = 0
for explanation_name, explanations in explanations_to_test.items():
  insertion_score = metric(explanations)
  insertion_scores.append((explanation_name, insertion_score))

pd.DataFrame(insertion_scores, columns=["Method name", "Insertion score (higher is better)"])

## **MuFidelity** Metric

Used to compute the fidelity correlation metric. This metric ensure there is a correlation between a random subset of pixels and their attribution score. For each random subset created, we set the pixels of the subset at a baseline state and obtain the prediction score. This metric measures the correlation between the drop in the score and the importance of the explanation.

**Higher is better**

---

Bhatt & al., Evaluating and Aggregating Feature-based Model Explanations (2020). https://arxiv.org/abs/2005.00631 (def. 3)

In [ ]:
from xplique.metrics import MuFidelity

metric = MuFidelity(model, X, Y, batch_size, nb_samples=25)
fidelity_scores = []

i = 0
for explanation_name, explanations in explanations_to_test.items():
  fidelity_score = metric(explanations)
  fidelity_scores.append((explanation_name, fidelity_score))

pd.DataFrame(fidelity_scores, columns=["Method name", "Fidelity score (higher is better)"])

## **Stability** Metric

Used to compute the average sensitivity metric (or stability). This metric ensure that close inputs with similar predictions yields similar explanations. For each inputs we randomly sample noise to add to the inputs and compute the explanation for the noisy inputs. We then get the average distance between the original explanations and the noisy explanations.

**Lower is better**

---

Bhatt & al., Evaluating and Aggregating Feature-based Model Explanations (2020). https://arxiv.org/abs/2005.00631 (def. 2)

In [ ]:
from xplique.metrics import AverageStability

metric = AverageStability(model, X[:20], Y[:20], batch_size, nb_samples=32)
stability_scores = []

i = 0
for explainer in explainers:
  explainer_name = explainer.__class__.__name__
  stability_score = metric(explainer)
  stability_scores.append((explainer_name, stability_score))

pd.DataFrame(stability_scores, columns=["Method name", "Stability score (lower is better)"])

# 🎉 Congratulations
**you now know how to use the Attribution Metric module of Xplique !**